# Red neuronal regresión con datos numéricos
En ese notebook usaremos una red neuronal de regresión para predecir el precio de los apartamentos sin tener en cuenta las imágenes, se hará con datos numéricos.

El primer paso será cargar dos datasets de train y test y hacer el escalado de características

## Carga de datos y escalado

In [1]:
import tensorflow as tf

tf.enable_eager_execution()

In [2]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())
print( tf.constant( 'Hellow from TensorFlow ' + tf.__version__ ) )

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5087621869623343280
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3063309926
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14484855071247566593
physical_device_desc: "device: 0, name: GeForce GTX 1650 with Max-Q Design, pci bus id: 0000:03:00.0, compute capability: 7.5"
]
tf.Tensor(b'Hellow from TensorFlow 1.15.0', shape=(), dtype=string)


In [3]:
import pandas as pd
import os
from sklearn import preprocessing

# Carga el dataset y devuelve un dataframe de Pandas
def load_airbnb_dataset(ruta,nombre):
    csv_path = os.path.join(ruta, nombre)
    return pd.read_csv(csv_path, sep=';')

In [4]:
copTrain = load_airbnb_dataset('datasets', 'trainFinal.csv')
copTest = load_airbnb_dataset('datasets', 'testFinal.csv')

# Separo la Y del resto de datos
dataPrecio = copTrain['Price']
dataSinPrecio = copTrain.drop(['Price'], axis=1, inplace=False)

dataPrecioTest = copTest['Price']
dataSinPrecioTest = copTest.drop(['Price'], axis=1, inplace=False)

y_train = dataPrecio.values
X_train = dataSinPrecio.values

y_test = dataPrecioTest.values
X_test = dataSinPrecioTest.values

feature_names = copTrain.columns[:]

# Obtener precio máximo en Train, y escalamos los precios de test y train en rango de [0, 1]
maxPrice = copTrain["Price"].max()
trainY = copTrain["Price"] / maxPrice
testY = copTest["Price"] / maxPrice

# Escalamos variables numéricas de train y test
scaler = preprocessing.StandardScaler().fit(X_test)
XtestScaled = scaler.transform(X_test)

scaler = preprocessing.StandardScaler().fit(X_train)
XtrainScaled = scaler.transform(X_train)


## Red neuronal
En este caso, usaremos una red neuronal (MLP) Multiplayer Perceptron

In [5]:
# Núemero de características
XtrainScaled.shape

(8262, 97)

Para calcular el nº de neuronas por capa oculta, uso la **Regla de la pirámide geométrica**.

Tenemos 97 características (97 neuronas de entrada).
Al ser un problema de regresión, tenemos una neurona de salida.

Dividiré por 3 capas ocultas de la siguiente forma:

r = raíz cuarta de 97/1
r = 3

h1 = 1 * 3^3 = 27

h2 = 1 * 3^2 = 9

h3 = 1 * 3 = 3

In [6]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model

def create_mlp(dim):
	# define our MLP network
	model = Sequential()
	model.add(Dense(27, input_dim=dim, activation="relu"))
	model.add(Dense(9, activation="relu"))
	model.add(Dense(3, activation="relu"))
	model.add(Dense(1, activation="linear"))
	
	return model

Using TensorFlow backend.


In [ ]:
model = models.create_mlp(XtrainScaled.shape[1])
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

# train the model
print("[INFO] training model...")
model.fit(trainX, trainY, validation_data=(testX, testY),
	epochs=200, batch_size=8)